In [98]:
import ccxt
cb = ccxt.coinbasepro({
   'apiKey': '...',
    'secret': '...',
    'urls': {
     'api': {
                'public': 'https://api-public.sandbox.pro.coinbase.com',
                'private': 'https://api-public.sandbox.pro.coinbase.com',
            },
    },    
    'password':'...'
})
cb.load_markets()
symbol = 'BTC/USD'
# print(cb.fetch_ticker('BTC/USD'))
# print(cb.fetch_balance())
# cb.create_market_buy_order (symbol, '0.01')
cb.fetch_my_trades(symbol)

[{'id': '21714222',
  'order': '24728796-41fc-45ee-b291-e545f37158fc',
  'info': {'created_at': '2020-12-30T23:50:28.296Z',
   'trade_id': 21714222,
   'product_id': 'BTC-USD',
   'order_id': '24728796-41fc-45ee-b291-e545f37158fc',
   'user_id': '5fe53030706b8613d795bd7e',
   'profile_id': '9a97b579-2c6a-4150-ba67-8363bcc767c5',
   'liquidity': 'T',
   'price': '28816.64000000',
   'size': '0.01000000',
   'fee': '1.4408320000000000',
   'side': 'buy',
   'settled': True,
   'usd_volume': '288.1664000000000000'},
  'timestamp': 1609372228296,
  'datetime': '2020-12-30T23:50:28.296Z',
  'symbol': 'BTC/USD',
  'type': None,
  'takerOrMaker': 'taker',
  'side': 'buy',
  'price': 28816.64,
  'amount': 0.01,
  'fee': {'cost': 1.440832, 'currency': 'USD', 'rate': 0.005},
  'cost': 288.1664}]

In [61]:
#!/usr/bin/env python
# -*- coding: utf-8; py-indent-offset:4 -*-
###############################################################################
#
# Copyright (C) 2017,2018 Ed Bartosh <bartosh@gmail.com>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
#
###############################################################################
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import sys

import backtrader as bt

class TestStrategy(bt.Strategy):
    def next(self):
        for data in self.datas:
            print('*' * 5, 'NEXT:', bt.num2date(data.datetime[0]), data._name, data.open[0],
                  data.high[0], data.low[0], data.close[0], data.volume[0],
                  bt.TimeFrame.getname(data._timeframe), len(data))
            if not self.getposition(data):
#                 self.buy(data, exectype=bt.Order.Limit, size=1, price=data.close[0])
                self.buy()

    def notify_order(self, order):
        print('*' * 5, "NOTIFY ORDER", order)

def runstrategy(argv):
    # Create a cerebro
    cerebro = bt.Cerebro()

    # Create data feeds
    data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                               timeframe=bt.TimeFrame.Ticks, compression=1)
    cerebro.adddata(data_ticks)

    # Add the strategy
    cerebro.addstrategy(TestStrategy)

    # Run the strategy
    cerebro.run()

if __name__ == '__main__':
    sys.exit(runstrategy(sys.argv))

***** NEXT: 2020-12-30 07:35:08.268002 btc_usd_tick 28315.21 28315.21 28315.21 28315.21 0.03339598 Tick 1
***** NOTIFY ORDER Ref: 605
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 1
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737789.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
***** NOTIFY ORDER Ref: 605
OrdType: 0
OrdType: Buy
Status: 7
Status: Margin
Size: 1
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737789.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: False
***** NEXT: 2020-12-30 07:35:08.268002 btc_usd_tick 28317.85 28317.85 28317.85 28317.85 0.19593661 Tick 2
***** NOTIFY ORDER Ref: 606
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 1
Price: None
Price Limit: None
TrailAmount: None
TrailPercent: None
ExecType: 0
ExecType: Market
CommInfo: None
End of Session: 737789.9999

KeyboardInterrupt: 

In [87]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)
    

import sys

import backtrader as bt

class TestStrategy(bt.Strategy):
    def next(self):
        for data in self.datas:
            print('*' * 5, 'NEXT:', bt.num2date(data.datetime[0]), data._name, data.open[0],
                  data.high[0], data.low[0], data.close[0], data.volume[0],
                  bt.TimeFrame.getname(data._timeframe), len(data))
            if not self.getposition(data):
#                 self.buy(data, exectype=bt.Order.Limit, size=1, price=data.close[0])
                self.buy(size=0.01)

    def notify_order(self, order):
        print('*' * 5, "NOTIFY ORDER", order)

def runstrategy(argv):
    # Create a cerebro
    cerebro = bt.Cerebro()
    
#     broker_config = {'urls': {'api': 'https://api-public.sandbox.pro.coinbase.com'},
#                     'apiKey': '0d18c023c2835e869f7772c2f90f491a',
#                     'secret': 'K7qCEIYBjFHVgFUEzbuWPH56+/gYqrsF+LIHTlmYknTIJDxDyCqL2UGoLROsB41cHQH6lvW6LpccbngbHNELhA==',
#                      'nonce': lambda: str(int(time.time() * 1000)),
#                      'password':'coinbase'
#                     }
    broker_config = {
        'apiKey': '0d18c023c2835e869f7772c2f90f491a',
        'secret': 'K7qCEIYBjFHVgFUEzbuWPH56+/gYqrsF+LIHTlmYknTIJDxDyCqL2UGoLROsB41cHQH6lvW6LpccbngbHNELhA==',
#         'urls': {
#             'api': {
#                 'public': 'https://api-public.sandbox.pro.coinbase.com',
#                 'private': 'https://api-public.sandbox.pro.coinbase.com',
#             },
#         },
#         'password':'coinbase',
    }

    # Create data feeds
    data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                               timeframe=bt.TimeFrame.Ticks, compression=1, config=broker_config)
    cerebro.adddata(data_ticks)

    broker = bt.brokers.CCXTBroker(exchange='coinbasepro', currency='USD', config=broker_config)
    cerebro.setbroker(broker)

    # Add the strategy
    cerebro.addstrategy(TestStrategy)

    # Run the strategy
    cerebro.run()

if __name__ == '__main__':
    sys.exit(runstrategy(sys.argv))

AuthenticationError: requires `apiKey`